# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, linked_variables: Dict[str, str] = {}) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negat

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables={})

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': {}}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables": {"a": [0.0, 1000.0], "b": [-1.0, 1.0]}, "constraints": {"e": ["LESS_THAN", 2.0], "f": ["GREATER_THAN", 0.0]}, "objectives": {"c": "MAXIMIZE", "d": "MINIMIZE"}, "constants": {"g": 1234}, "linked_variables": {}}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,902.930442,0.393160,1234,903.323602,902.537282,1806.647205,1805.074563
6,605.412241,-0.018832,1234,605.393409,605.431073,1210.786818,1210.862146
7,505.282752,0.390182,1234,505.672934,504.892570,1011.345868,1009.785141
8,948.216455,-0.685834,1234,947.530621,948.902288,1895.061242,1897.804576
9,909.501838,0.953692,1234,910.455530,908.548147,1820.911060,1817.096294
10,591.766430,-0.598144,1234,591.168286,592.364573,1182.336572,1184.729147
11,876.388416,0.150928,1234,876.539344,876.237488,1753.078688,1752.474976
12,574.215562,-0.717009,1234,573.498553,574.932571,1146.997106,1149.865142
13,933.380630,0.367667,1234,933.748298,933.012963,1867.496596,1866.025926
14,594.963646,-0.089575,1234,594.874070,595.053221,1189.748141,1190.106442


In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-903.323602,902.537282
6,-605.393409,605.431073
7,-505.672934,504.892570
8,-947.530621,948.902288
9,-910.455530,908.548147
10,-591.168286,592.364573
11,-876.539344,876.237488
12,-573.498553,574.932571
13,-933.748298,933.012963
14,-594.874070,595.053221


In [13]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-903.323602,902.537282
6,-605.393409,605.431073
7,-505.672934,504.892570
8,-947.530621,948.902288
9,-910.455530,908.548147
10,-591.168286,592.364573
11,-876.539344,876.237488
12,-573.498553,574.932571
13,-933.748298,933.012963
14,-594.874070,595.053221


In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-903.32360225,  902.53728163],
       [-605.39340921,  605.43107291],
       [-505.67293382,  504.89257042],
       [-947.53062121,  948.90228823],
       [-910.4555301 ,  908.54814675],
       [-591.16828596,  592.36457346],
       [-876.53934393,  876.2374881 ],
       [-573.49855304,  574.93257089],
       [-933.74829793,  933.01296294],
       [-594.87407034,  595.05322099]])

In [15]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,1804.647205,-1805.074563
6,1208.786818,-1210.862146
7,1009.345868,-1009.785141
8,1893.061242,-1897.804576
9,1818.911060,-1817.096294
10,1180.336572,-1184.729147
11,1751.078688,-1752.474976
12,1144.997106,-1149.865142
13,1865.496596,-1866.025926
14,1187.748141,-1190.106442


In [16]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
